In [ ]:
!pip install onnxruntime

In [32]:
import onnxruntime
import cv2
import numpy as np
import time

In [33]:

def load_class_names(namesfile):
    class_names = []
    with open(namesfile, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.rstrip()
        class_names.append(line)
    return class_names

In [34]:

def nms_cpu(boxes, confs, nms_thresh=0.5, min_mode=False):
    # print(boxes.shape)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    areas = (x2 - x1) * (y2 - y1)
    order = confs.argsort()[::-1]

    keep = []
    while order.size > 0:
        idx_self = order[0]
        idx_other = order[1:]

        keep.append(idx_self)

        xx1 = np.maximum(x1[idx_self], x1[idx_other])
        yy1 = np.maximum(y1[idx_self], y1[idx_other])
        xx2 = np.minimum(x2[idx_self], x2[idx_other])
        yy2 = np.minimum(y2[idx_self], y2[idx_other])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h

        if min_mode:
            over = inter / np.minimum(areas[order[0]], areas[order[1:]])
        else:
            over = inter / (areas[order[0]] + areas[order[1:]] - inter)

        inds = np.where(over <= nms_thresh)[0]
        order = order[inds + 1]
    
    return np.array(keep)

In [35]:

def post_processing(img, conf_thresh, nms_thresh, output):
    # anchors = [12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401]
    # num_anchors = 9
    # anchor_masks = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    # strides = [8, 16, 32]
    # anchor_step = len(anchors) // num_anchors

    # [batch, num, 1, 4]
    box_array = output[0]
    # [batch, num, num_classes]
    confs = output[1]

    t1 = time.time()

    if type(box_array).__name__ != 'ndarray':
        box_array = box_array.cpu().detach().numpy()
        confs = confs.cpu().detach().numpy()

    num_classes = confs.shape[2]

    # [batch, num, 4]
    box_array = box_array[:, :, 0]

    # [batch, num, num_classes] --> [batch, num]
    max_conf = np.max(confs, axis=2)
    max_id = np.argmax(confs, axis=2)

    t2 = time.time()

    bboxes_batch = []
    for i in range(box_array.shape[0]):
       
        argwhere = max_conf[i] > conf_thresh
        l_box_array = box_array[i, argwhere, :]
        l_max_conf = max_conf[i, argwhere]
        l_max_id = max_id[i, argwhere]

        bboxes = []
        # nms for each class
        for j in range(num_classes):

            cls_argwhere = l_max_id == j
            ll_box_array = l_box_array[cls_argwhere, :]
            ll_max_conf = l_max_conf[cls_argwhere]
            ll_max_id = l_max_id[cls_argwhere]

            keep = nms_cpu(ll_box_array, ll_max_conf, nms_thresh)
            
            if (keep.size > 0):
                ll_box_array = ll_box_array[keep, :]
                ll_max_conf = ll_max_conf[keep]
                ll_max_id = ll_max_id[keep]

                for k in range(ll_box_array.shape[0]):
                    bboxes.append([ll_box_array[k, 0], ll_box_array[k, 1], ll_box_array[k, 2], ll_box_array[k, 3], ll_max_conf[k], ll_max_conf[k], ll_max_id[k]])
        
        bboxes_batch.append(bboxes)

    t3 = time.time()

    # print('-----------------------------------')
    # print('       max and argmax : %f' % (t2 - t1))
    # print('                  nms : %f' % (t3 - t2))
    # print('Post processing total : %f' % (t3 - t1))
    # print('-----------------------------------')
    
    return bboxes_batch

In [10]:
onnx_full_path = detection_model["active_version_id"] + ".onnx"

In [27]:
session = onnxruntime.InferenceSession(onnx_full_path)

IN_IMAGE_H = session.get_inputs()[0].shape[2]
IN_IMAGE_W = session.get_inputs()[0].shape[3]

image_src = cv2.imread("bus-car-person.jpg")

resized = cv2.resize(image_src, (IN_IMAGE_W, IN_IMAGE_H), interpolation=cv2.INTER_LINEAR)
img_in = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
img_in = np.transpose(img_in, (2, 0, 1)).astype(np.float32)
img_in = np.expand_dims(img_in, axis=0)
img_in /= 255.0

input_name = session.get_inputs()[0].name

outputs = session.run(None, {input_name: img_in})

boxes = post_processing(img_in, 0.4, 0.6, outputs)

In [30]:
labels_file = detection_model["active_version_id"] + ".names"
class_names = load_class_names(labels_file)

In [31]:
for box in boxes[0]:
    print(box)
   
    print(class_names[box[6]])


[-0.0074151456, 0.007669747, 0.3948107, 0.9739917, 0.9312626, 0.9312626, 0]
person
[0.63776255, 0.01903963, 0.99418163, 0.9847417, 0.9092924, 0.9092924, 0]
person
[0.29356784, 0.22734743, 0.33088827, 0.31162006, 0.8798908, 0.8798908, 2]
car
[0.77916926, 0.18809426, 0.8264917, 0.3314556, 0.5911089, 0.5911089, 2]
car
[0.33751625, -0.00433293, 0.76922274, 0.6129248, 0.95931864, 0.95931864, 5]
bus


In [48]:
def yolo_predict(model, img):
    onnx_full_path = model["active_version_id"] + ".onnx"
    session = onnxruntime.InferenceSession(onnx_full_path)

    IN_IMAGE_H = session.get_inputs()[0].shape[2]
    IN_IMAGE_W = session.get_inputs()[0].shape[3]

    image_src = cv2.imread(img)

    resized = cv2.resize(image_src, (IN_IMAGE_W, IN_IMAGE_H), interpolation=cv2.INTER_LINEAR)
    img_in = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
    img_in = np.transpose(img_in, (2, 0, 1)).astype(np.float32)
    img_in = np.expand_dims(img_in, axis=0)
    img_in /= 255.0

    input_name = session.get_inputs()[0].name

    outputs = session.run(None, {input_name: img_in})

    boxes = post_processing(img_in, 0.4, 0.6, outputs)
    
    labels_file = model["active_version_id"] + ".names"
    class_names = load_class_names(labels_file)
   
    
    for box in boxes[0]:
       
        print(box)

        print(class_names[box[6]])
        print("------")
        

# SeeMe.ai quick guide Object Detection at the edge

# 1. Setup

## 1.1 Where to run your Jupyter Notebooks

Use your preferred platfrom to run your [Jupyter Notebooks](https://jupyter.org/): 

* [Paperspace](https://paperspace.com)
* [Google Colab](https://colab.research.google.com/)
* [The official Fast.ai Docker images](https://hub.docker.com/u/fastdotai)
* [Fast.ai Docker from SeeMe.ai](https://hub.docker.com/repository/docker/seemeai/fastai)
* [Azure](https://azure.microsoft.com)
* ...

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.9.0


## 1.2 SeeMe.ai Python SDK

Install the [SeeMe SDK](https://pypi.org/project/seeme/) from the command line:

```bash
$ pip install --upgrade --no-cache-dir seeme
```

or in your Notebook:

     |████████████████████████████████| 4.8 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 9.6 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
# Install the SeeMe SDK from your notebook.
!pip install --upgrade --no-cache-dir seeme

You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


# 2. Create a client

In [38]:
from seeme import Client

In [39]:
cl = Client(username="janvdp", apikey="a8b4f486-87a8-47dd-beaa-596086cfc146")

In [40]:
detection_model = cl.get_model('5d3dbde4-0600-43c6-9a37-d5cd259c0267')

In [41]:
cl.download_active_model(detection_model, assetType="onnx")

In [42]:
cl.download_active_model(detection_model, assetType="names")

In [49]:
yolo_predict(detection_model, "bus-car-person.jpg")

[-0.0074151456, 0.007669747, 0.3948107, 0.9739917, 0.9312626, 0.9312626, 0]
person
------
[0.63776255, 0.01903963, 0.99418163, 0.9847417, 0.9092924, 0.9092924, 0]
person
------
[0.29356784, 0.22734743, 0.33088827, 0.31162006, 0.8798908, 0.8798908, 2]
car
------
[0.77916926, 0.18809426, 0.8264917, 0.3314556, 0.5911089, 0.5911089, 2]
car
------
[0.33751625, -0.00433293, 0.76922274, 0.6129248, 0.95931864, 0.95931864, 5]
bus
------
